In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import os
from pathlib import Path
from keras import models

2025-07-10 19:27:19.046457: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-10 19:27:19.062858: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752164839.082966    6564 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752164839.088410    6564 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752164839.102117    6564 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

AttributeError: module 'inspect' has no attribute 'ArgSpec'

In [ ]:
SAMPLING_RATE = 9000 # Частота дискретизации 
WINDOW_SIZE = 4096 # Размер фрейма для дробления исходных данных   
STEP = 2048 # Шаг фрейма


NPERSEG = 512  # Размер фрейма для создания спектрограммы            
NOVERLAP = NPERSEG - 1 #коэф наложения фрейма

DPI = 100 # Количество точек на дюйм для изображения
IMG_HEIGHT = 257   # Высота изображения 
IMG_WIDTH = 3585 # Ширина изображения
CLASS_NAMES = ['defect', 'normal'] # Классы модели, важен порядок (смотри train.ipynb)

In [ ]:
def create_spectrogram_from_chunk_normalized(chunk_data, fs):
    """
    Создает нормализованную спектрограмму из фрагмента данных в памяти, не сохраняя на диск.
    Возвращает изображение в виде numpy-массива.
    """
    noverlap = NPERSEG - 1
    frequencies, times, Zxx = signal.stft(
        chunk_data, 
        fs=fs, 
        nperseg=NPERSEG, 
        noverlap=NOVERLAP, 
        boundary=None
    )
    
    # Получаем модуль спектрограммы
    Zxx_abs = np.abs(Zxx)
    
    # Нормализация данных от 0 до 1
    min_val = np.min(Zxx_abs)
    max_val = np.max(Zxx_abs)
    if max_val > min_val:
        Zxx_normalized = (Zxx_abs - min_val) / (max_val - min_val)
    else:
        Zxx_normalized = np.zeros_like(Zxx_abs)
    
    height, width = Zxx_normalized.shape
    dpi = DPI
    fig_size = (width / dpi, height / dpi)
    
    fig = plt.figure(figsize=fig_size, dpi=dpi)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    
    ax.imshow(Zxx_normalized, cmap='gray', origin='lower', aspect='auto')
    
    fig.canvas.draw()
    
    buf = fig.canvas.buffer_rgba()
    img_array = np.frombuffer(buf, dtype=np.uint8).reshape(fig.canvas.get_width_height()[::-1] + (4,))
    img_array_rgb = img_array[:, :, :3]
    
    plt.close(fig)
    return img_array_rgb

In [ ]:
def create_spectrogram_from_chunk(chunk_data, fs):
    """
    Создает спектрограмму из фрагмента данных в памяти, не сохраняя на диск.
    Возвращает изображение в виде numpy-массива.
    """
    noverlap = NPERSEG - 1
    
    fig = plt.figure(figsize=(IMG_WIDTH/DPI, IMG_HEIGHT/DPI), dpi=DPI)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)

    frequencies, times, Zxx = signal.stft(chunk_data, fs=fs, nperseg=NPERSEG, noverlap=noverlap)
    ax.pcolormesh(times, frequencies, np.log(np.abs(Zxx) + 1e-9), cmap='gray')
    
    fig.canvas.draw()
    
    buf = fig.canvas.buffer_rgba()
    img_array = np.frombuffer(buf, dtype=np.uint8).reshape(fig.canvas.get_width_height()[::-1] + (4,))

    img_array_rgb = img_array[:, :, :3]
   
    
    plt.close(fig)
    return img_array_rgb 

In [ ]:
def predict_vibration_state(csv_path, model_path):
    """
    Загружает CSV файл, обрабатывает его и делает предсказание с помощью модели.
    """
    print(f"Загрузка модели из файла: {model_path}")
    try:
        model = models.load_model(model_path)
    except Exception as e:
        print(f"Ошибка при загрузке модели: {e}")
        return

    print(f"Чтение и обработка файла: {csv_path}")
    try:
        df = pd.read_csv(csv_path)
        if 'vibration' not in df.columns:
            print("Ошибка: в CSV файле отсутствует колонка 'vibration'.")
            return
        signal_data = df['vibration'].values
    except Exception as e:
        print(f"Ошибка при чтении CSV файла: {e}")
        return

    predictions = []

    for i in range(0, len(signal_data) - WINDOW_SIZE + 1, STEP):
        chunk = signal_data[i : i + WINDOW_SIZE]

        spectrogram_img = create_spectrogram_from_chunk_normalized(chunk, SAMPLING_RATE)
        
        img_tensor = tf.expand_dims(spectrogram_img, 0)

        prediction = model.predict(img_tensor, verbose=0)
        
        score = tf.nn.sigmoid(prediction[0][0])
        
        predicted_class_index = 1 if score > 0.5 else 0
        predictions.append(CLASS_NAMES[predicted_class_index])

    if not predictions:
        print("Не удалось создать ни одного фрагмента для анализа. Файл слишком короткий.")
        return

    print(f"\nАнализ завершен. Всего проанализировано фрагментов: {len(predictions)}")
    
    defect_count = predictions.count('defect')
    normal_count = predictions.count('normal')
    
    print(f"  - Фрагментов 'normal': {normal_count}")
    print(f"  - Фрагментов 'defect': {defect_count}")
    
    if defect_count > 0:
        final_verdict = "DEFECT"
        print("\Результат: Defect")
    else:
        final_verdict = "NORMAL"
        print("\Результат: Norm")
        
    return final_verdict

In [ ]:
PATH_TO_NEW_CSV = 'csv_cwru_data/defect/B007_0.csv'
PATH_TO_MODEL = 'vibration_classifier_model_SELF_1024_NORM_e3.keras'

if not os.path.exists(PATH_TO_NEW_CSV):
    print(f"Файл не найден: {PATH_TO_NEW_CSV}. Укажите правильный путь.")
elif not os.path.exists(PATH_TO_MODEL):
    print(f"Модель не найдена: {PATH_TO_MODEL}. Укажите правильный путь.")
else:
    predict_vibration_state(PATH_TO_NEW_CSV, PATH_TO_MODEL)

Загрузка модели из файла: vibration_classifier_model_SELF_1024_NORM_e3.keras
Ошибка при загрузке модели: File not found: filepath=vibration_classifier_model_SELF_1024_NORM_e3.keras. Please ensure the file is an accessible `.keras` zip file.


In [ ]:
SAPSAN_FIRST_CSV_FOLDER = "csv_cwru_data/defect/"
PATH_TO_MODEL = 'vibration_classifier_model_SELF_512_NORM_e3.keras'

for file in Path(SAPSAN_FIRST_CSV_FOLDER).iterdir():
    if file.is_file():
        print(f"Prediction file {file}: ")
        predict_vibration_state(file, PATH_TO_MODEL)

Prediction file csv_cwru_data/defect/B007_0.csv: 
Загрузка модели из файла: vibration_classifier_model_SELF_512_NORM_e3.keras
Ошибка при загрузке модели: File not found: filepath=vibration_classifier_model_SELF_512_NORM_e3.keras. Please ensure the file is an accessible `.keras` zip file.
Prediction file csv_cwru_data/defect/B028_1.csv: 
Загрузка модели из файла: vibration_classifier_model_SELF_512_NORM_e3.keras
Ошибка при загрузке модели: File not found: filepath=vibration_classifier_model_SELF_512_NORM_e3.keras. Please ensure the file is an accessible `.keras` zip file.
Prediction file csv_cwru_data/defect/B021_0.csv: 
Загрузка модели из файла: vibration_classifier_model_SELF_512_NORM_e3.keras
Ошибка при загрузке модели: File not found: filepath=vibration_classifier_model_SELF_512_NORM_e3.keras. Please ensure the file is an accessible `.keras` zip file.
Prediction file csv_cwru_data/defect/B028_2.csv: 
Загрузка модели из файла: vibration_classifier_model_SELF_512_NORM_e3.keras
Ошибка 

In [ ]:
SAPSAN_SECOND_CSV_FOLDER = "csv_sapsans_data/second"
PATH_TO_MODEL = 'models/self/vibration_classifier_model_SELF_256_e25.keras'

for file in Path(SAPSAN_SECOND_CSV_FOLDER).iterdir():
    if file.is_file():
        print(f"Prediction file {file}: ")
        predict_vibration_state(file, PATH_TO_MODEL)

FileNotFoundError: [Errno 2] No such file or directory: 'csv_sapsans_data/second'